In [ ]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path.append("code")
import numpy as np
import matplotlib.pyplot as plt
from keras_tqdm import TQDMNotebookCallback
from npm3dGenerator_tree import NPM3DGenerator, NPM3DGenerator_full
from models import build_point_net, save_model, load_model
import utils_training as u_t

In [ ]:
training_dataset = NPM3DGenerator(paths_to_keep = [0, 1, 2], sample_uniformly_from_classes = True)
validation_dataset = NPM3DGenerator(paths_to_keep = [0], sample_uniformly_from_classes = True)
print(training_dataset)

In [ ]:
miou_metric = u_t.mIoU(training_dataset.n_classes)

#model = build_point_net(input_shape = (training_dataset.n_points, training_dataset.n_channels), output_shape = training_dataset.n_classes)
model = load_model("models/trained_{}_{}_{}".format(training_dataset.n_points, training_dataset.n_channels, training_dataset.n_classes))
model.compile("adam", u_t.my_categorical_crossentropy, metrics = ["accuracy", miou_metric.miou])
#model.summary()

In [ ]:
n_epochs = 2
history = model.fit_generator(training_dataset, epochs = n_epochs,
                              validation_data = validation_dataset,
                              class_weight = training_dataset.class_weight,
                              verbose = 0, callbacks=[TQDMNotebookCallback(leave_inner=True, leave_outer=True)])
u_t.plot_history(history)

In [ ]:
model_name = "models/trained_{}_{}_{}_2x10epochs_plus_6epochs_uniform_sample_fixingbug".format(training_dataset.n_points, training_dataset.n_channels, training_dataset.n_classes)
save_model(model, model_name)

In [ ]:
test_dataset = NPM3DGenerator(input_dir = "../Benchmark_MVA/test", train = False)
#model = load_model("models/trained_{}_{}_{}".format(test_dataset.n_points, test_dataset.n_channels, test_dataset.n_classes))
probas, predictions = test_dataset.predict_point_cloud(model)
try:np.savetxt('predictions_{}.txt'.format(model_name.split("/")[-1]), predictions, fmt='%d')
except:np.savetxt('predictions.txt', predictions, fmt='%d')

plt.hist(predictions)
plt.show()

plt.figure(figsize = (15, 7))
for i in range(test_dataset.n_classes):
    plt.subplot(231 + i)
    plt.title(test_dataset.class_dict[i + 1])
    plt.hist(np.max(probas[predictions == 1 + i], axis = -1), normed = True)
plt.show()